# Testing extracting OSM data using Osmium

In [11]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

# pip install osmium
import osmium, logging
import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

import time

In [12]:
# set file
some_file = './sri_lanka.osm.pbf'

In [13]:
in_folder = "./osm"

In [14]:
traffic_csv_1233302 = os.path.join(in_folder, "1233302-Asia-Colombo.csv")
traffic_1233302 = pd.read_csv(traffic_csv_1233302, header = None)
traffic_1233302

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,102738212,3279663552,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
1,1093698450,1093697790,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
2,1093698671,1093697934,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
3,1093798472,1093799200,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
4,1093798791,1093799519,34,34,34,34,34,34,34,34,...,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467858,937250922,2334155507,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
467859,98553797,6409372992,50,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
467860,98554190,3814750237,47,47,47,47,47,47,47,47,...,47,47,47,47,47,47,47,47,47,47
467861,98557759,2155638061,36,36,36,36,36,36,36,36,...,36,36,36,36,36,36,36,36,36,36


In [15]:
traffic_csv_1233303 = os.path.join(in_folder, "1233303-Asia-Colombo.csv")
traffic_1233303 = pd.read_csv(traffic_csv_1233303, header = None)
traffic_1233303

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1147027495,3318178916,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
1,1147029008,3333484939,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
2,1147030124,3318163467,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
3,1675585654,1675585568,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
4,1675667479,5787866342,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9921,6495299720,6495344002,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
9922,6495398243,6495398223,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
9923,6899782766,3317976522,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
9924,7082570748,7082570747,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66


## Extracting highways and nodes using Osmium

In [16]:
start_time = time.time()

wkbfab = osmium.geom.WKBFactory()

# extract highways
class HighwayExtractor(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self) 
        self.nodes = []
        #self.raw_h = []
        self.highways = []
        self.broken_highways = []
        self.total = 0
        self.num_nodes = 0
    
    def way(self, w):
        #self.raw_h.append(w)
        try:
            nodes = [x.ref for x in w.nodes]
            wkb = wkbfab.create_linestring(w)
            shp = wkblib.loads(wkb, hex=True)
            if 'highway' in w.tags:
                info = [w.id, nodes, shp, w.tags['highway']]
                self.highways.append(info)
        except:
            print('hit exception')
            nodes = [x for x in w.nodes if x.location.valid()]
            if len(nodes) > 1:
                shp = LineString([Point(x.location.x, x.location.y) for x in nodes])
                info = [w.id, nodes, shp, w.tags['highway']]
                self.highways.append(info)
            else:
                self.broken_highways.append(w)
            logging.warning("Error Processing OSM Way %s" % w.id)
            
        
h = HighwayExtractor()
h.apply_file(some_file, locations=True)
print(len(h.highways))
print(len(h.broken_highways))

end_time = time.time()
print(end_time - start_time)

228036
0
265.0243990421295


In [17]:
#h.nodes

In [18]:
#h.highways

In [19]:
h.highways[1]

[4860428,
 [31363742,
  31363743,
  5379799622,
  3726382944,
  3726382945,
  3776248086,
  31363744,
  3726382943,
  31363745,
  3726382942,
  3726382941,
  31363746,
  31363747,
  31363748,
  31363749,
  5183847099,
  31363051],
 'residential']

In [20]:
#h.highways[2][2].length

In [21]:
#list(h.highways[0][2].coords)

In [22]:
#list(h.highways[2][2].coords)[2]

In [10]:
start_time = time.time()

all_nodes = []
all_edges = []

for x in h.highways:
    for n_idx in range(0, (len(x[1]) - 1)):
        try:
            osm_id_from = x[1][n_idx].ref
        except:
            osm_id_from = x[1][n_idx]
        try:
            osm_id_to   = x[1][n_idx+1].ref
        except:
            osm_id_to   = x[1][n_idx+1]
        try:
            osm_coords_from = list(x[2].coords)[n_idx]
            #print(osm_coords_from[0])
            #create a node
            all_nodes.append([osm_id_from, { 'x' : osm_coords_from[0], 'y' : osm_coords_from[1] }])
            osm_coords_to = list(x[2].coords)[n_idx+1]
            #print(n_idx)
            #print(len(x[1]) - 1)
            if n_idx == (len(x[1]) - 2):
                #print('last element')
                #print(osm_coords_to)
                #create a node
                all_nodes.append([osm_id_to, { 'x' : osm_coords_to[0], 'y' : osm_coords_to[1]} ])
            edge = LineString([osm_coords_from, osm_coords_to])
            attr = {'osm_id':x[0], 'Wkt':edge, 'length':edge.length, 'infra_type':x[3]}
            #Create an edge from the list of nodes in both directions
            all_edges.append([osm_id_from, osm_id_to, attr])
            all_edges.append([osm_id_to, osm_id_from, attr])
        except:
            logging.warning(f"Error adding edge between nodes {osm_id_from} and {osm_id_to}")

end_time = time.time()
print(end_time - start_time)

256.35570311546326


In [11]:
#all_nodes[:10]

In [12]:
#all_edges[:10]

In [13]:
G = nx.MultiDiGraph()
G.add_nodes_from(all_nodes)
G.add_edges_from(all_edges)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [14]:
gn.example_edge(G)

(1650104033, 6236632580, {'osm_id': 4860427, 'Wkt': <shapely.geometry.linestring.LineString object at 0x12bb7c790>, 'length': 8.823270368842839e-05, 'infra_type': 'trunk'})


In [15]:
gn.example_node(G)

(1650104033, {'x': 80.5488946, 'y': 5.9451973})


In [16]:
edges_gdf = gn.edge_gdf_from_graph(G)

In [17]:
len(edges_gdf)

8650526

In [18]:
nodes_gdf = gn.node_gdf_from_graph(G)

## Mapbox Traffic Data

In [19]:
in_folder = "./osm"
traffic_csv = os.path.join(in_folder, "1233300-Asia-Colombo.csv")

In [20]:
# read in the traffic_csv
traffic = pd.read_csv(traffic_csv, header = None)
traffic.head()

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40


In [62]:
traffic

,0,1,2,3,4,5,6,7,8,9,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,1148494884,4177608798,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
1,1148495298,4137314867,57,57,57,57,57,57,57,57,...,57,57,57,57,57,57,57,57,57,57
2,1242700523,6537570627,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
3,1242730766,3377418986,46,46,46,46,46,46,46,46,...,46,46,46,46,46,46,46,46,46,46
4,1243299175,3805435746,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24246,849231590,2920467254,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
24247,901010011,2058634444,47,47,47,47,47,47,47,47,...,47,47,47,47,47,47,47,47,47,47
24248,901010134,3377392482,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
24249,904233206,1242795139,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24


In [21]:
def get_speeds(x):
    ''' Return Min, Max, and Mean speed '''
    x_vals = x[2:]
    return([min(x_vals), max(x_vals), np.mean(x_vals)]) #, np.argmax(x_vals)
    
traffic_vals = traffic.apply(lambda x: get_speeds(x), axis = 1, result_type = "expand")
traffic_vals.columns = ['min_speed','max_speed','mean_speed']

In [22]:
traffic_simplified = traffic.loc[:,[0,1]]
traffic_simplified.columns = ['FROM_NODE', "TO_NODE"]
traffic_simplified = traffic_simplified.join(traffic_vals)

In [23]:
traffic_simplified.head()

,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed
0,1148494884,4177608798,31.0,38.0,36.866071
1,1148495298,4137314867,57.0,57.0,57.000000
2,1242700523,6537570627,60.0,60.0,60.000000
3,1242730766,3377418986,46.0,46.0,46.000000
4,1243299175,3805435746,40.0,40.0,40.000000


# Add the traffic speeds to the edges

In [24]:
edges_gdf.loc[edges_gdf['stnode'] == 4965331097]

,stnode,endnode,osm_id,Wkt,infra_type,length,geometry


In [25]:
traffic_simplified.loc[traffic_simplified['FROM_NODE'] == 4965331097]

,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed


In [26]:
attributed_edges = edges_gdf.merge(traffic_simplified, left_on=['stnode','endnode'], right_on=['FROM_NODE','TO_NODE'])

In [27]:
attributed_edges

,stnode,endnode,osm_id,Wkt,infra_type,length,geometry,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed
0,1685598122,7065885049,44631879,"LINESTRING (80.5035486 8.404011199999999, 80.5...",primary,0.001273,"LINESTRING (80.50348 8.40528, 80.50355 8.40401)",1685598122,7065885049,69.0,69.0,69.000000
1,1685598632,1685598122,44631879,"LINESTRING (80.50348219999999 8.4052828, 80.50...",primary,0.000528,"LINESTRING (80.50346 8.40581, 80.50348 8.40528)",1685598632,1685598122,69.0,69.0,69.000000
2,7085359329,1685598632,44631879,"LINESTRING (80.5034601 8.405810300000001, 80.5...",primary,0.003108,"LINESTRING (80.50327 8.40891, 80.50346 8.40581)",7085359329,1685598632,57.0,67.0,66.898313
3,1685598304,7085359329,44631879,"LINESTRING (80.5032736 8.408912600000001, 80.5...",primary,0.000168,"LINESTRING (80.50326 8.40908, 80.50327 8.40891)",1685598304,7085359329,57.0,67.0,66.898313
4,7085359364,1685598304,44631879,"LINESTRING (80.5032635 8.4090799, 80.50288 8.4...",primary,0.007915,"LINESTRING (80.50288 8.41699, 80.50326 8.40908)",7085359364,1685598304,57.0,67.0,66.898313
...,...,...,...,...,...,...,...,...,...,...,...,...
24244,3377430317,3377430307,760091033,LINESTRING (81.24577119999999 8.44720049999999...,trunk,0.000189,"LINESTRING (81.24577 8.44720, 81.24559 8.44727)",3377430317,3377430307,35.0,65.0,48.869048
24245,3377430307,3377430317,760091033,LINESTRING (81.24577119999999 8.44720049999999...,trunk,0.000189,"LINESTRING (81.24559 8.44727, 81.24577 8.44720)",3377430307,3377430317,35.0,65.0,47.520833
24246,3377430307,1723379502,760091033,"LINESTRING (81.2455942 8.4472668, 81.245334099...",trunk,0.000270,"LINESTRING (81.24559 8.44727, 81.24533 8.44734)",3377430307,1723379502,35.0,65.0,48.869048
24247,1723379502,3377430307,760091033,"LINESTRING (81.2455942 8.4472668, 81.245334099...",trunk,0.000270,"LINESTRING (81.24533 8.44734, 81.24559 8.44727)",1723379502,3377430307,35.0,65.0,47.520833


In [52]:
# For a nx.MultiGraph you need an extra index going from 0 to n where n is the number of edges between the two nodes.
G[1723379500][1723379502][0]['mapbox_traffic_speed'] = 55

In [49]:
G[1723379502][1723379500]

AtlasView({0: {'osm_id': 760091033, 'Wkt': <shapely.geometry.linestring.LineString object at 0x296fbdc90>, 'length': 0.00039926492456985924, 'infra_type': 'trunk'}})

In [50]:
G[1723379500][1723379502]

AtlasView({0: {'osm_id': 760091033, 'Wkt': <shapely.geometry.linestring.LineString object at 0x296fbdc90>, 'length': 0.00039926492456985924, 'infra_type': 'trunk', 'mapbox_traffic_speed': 55}})

In [54]:
def add_edge_attribute(x):
    #print(x['osm_id'])
    #print(int(x['mean_speed']))
    G[x['stnode']][x['endnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    G[x['endnode']][x['stnode']][0]['mapbox_traffic_speed'] = int(x['mean_speed'])
    
attributed_edges.apply(lambda x: add_edge_attribute(x), axis = 1)

0        None
1        None
2        None
3        None
4        None
         ... 
24244    None
24245    None
24246    None
24247    None
24248    None
Length: 24249, dtype: object

In [55]:
G[1723379500][1723379502]

AtlasView({0: {'osm_id': 760091033, 'Wkt': <shapely.geometry.linestring.LineString object at 0x296fbdc90>, 'length': 0.00039926492456985924, 'infra_type': 'trunk', 'mapbox_traffic_speed': 48}})

# Export edges as shapefile to visualize

In [56]:
edge_gdf_w_traffic = gn.edge_gdf_from_graph(G)

In [57]:
edge_gdf_w_traffic

,stnode,endnode,length,infra_type,osm_id,Wkt,mapbox_traffic_speed,geometry
0,1650104033,6236632580,0.000088,trunk,4860427,"LINESTRING (80.5488946 5.9451973, 80.548912 5....",NaN,"LINESTRING (80.54889 5.94520, 80.54891 5.94511)"
1,1650104033,970058024,0.000145,trunk,152177791,"LINESTRING (80.5488725 5.9453403, 80.5488946 5...",NaN,"LINESTRING (80.54889 5.94520, 80.54887 5.94534)"
2,6236632580,1650104033,0.000088,trunk,4860427,"LINESTRING (80.5488946 5.9451973, 80.548912 5....",NaN,"LINESTRING (80.54891 5.94511, 80.54889 5.94520)"
3,6236632580,6236632579,0.000047,trunk,4860427,"LINESTRING (80.548912 5.9451108, 80.5489254 5....",NaN,"LINESTRING (80.54891 5.94511, 80.54893 5.94507)"
4,6236632579,6236632580,0.000047,trunk,4860427,"LINESTRING (80.548912 5.9451108, 80.5489254 5....",NaN,"LINESTRING (80.54893 5.94507, 80.54891 5.94511)"
...,...,...,...,...,...,...,...,...
8650521,7370017051,7370017050,0.000104,service,788282099,"LINESTRING (80.64003630000001 7.3823547, 80.64...",NaN,"LINESTRING (80.64004 7.38235, 80.64005 7.38225)"
8650522,7370017050,7370017051,0.000104,service,788282099,"LINESTRING (80.64003630000001 7.3823547, 80.64...",NaN,"LINESTRING (80.64005 7.38225, 80.64004 7.38235)"
8650523,7370017050,7370017049,0.000188,service,788282099,"LINESTRING (80.6400508 7.3822515, 80.640024100...",NaN,"LINESTRING (80.64005 7.38225, 80.64002 7.38207)"
8650524,7370017049,7370017050,0.000188,service,788282099,"LINESTRING (80.6400508 7.3822515, 80.640024100...",NaN,"LINESTRING (80.64002 7.38207, 80.64005 7.38225)"


In [60]:
# Therefore reduce number of columns, cannot have more than one column with geometry if saving to shapefile
edge_gdf_w_traffic_export = edge_gdf_w_traffic[['stnode','endnode','length','infra_type','osm_id','mapbox_traffic_speed','geometry']]

In [61]:
edge_gdf_w_traffic_export.to_file(driver = 'ESRI Shapefile', filename = './sri_lanka_hwy_w_traffic.shp')